In [1]:
from bs4 import BeautifulSoup
import requests
import openpyxl
import re
from openpyxl import load_workbook

In [2]:
#tables parser for mp staff function
import os
import requests
import urllib
import math
import copy
import pandas as pd	
import numpy as np
from bs4 import BeautifulSoup 

class html_tables(object):
    
    def __init__(self, url):
        
        self.url      = url
        self.r        = requests.get(self.url)
        self.url_soup = BeautifulSoup(self.r.text)
        
    def read(self):
        
        self.tables      = []
        self.tables_html = self.url_soup.find_all("table")
        
        # Parse each table
        for n in range(0, len(self.tables_html)):
            
            n_cols = 0
            n_rows = 0
            
            for row in self.tables_html[n].find_all("tr"):
                col_tags = row.find_all(["td", "th"])
                if len(col_tags) > 0:
                    n_rows += 1
                    if len(col_tags) > n_cols:
                        n_cols = len(col_tags)
            
            # Create dataframe
            df = pd.DataFrame(index = range(0, n_rows), columns = range(0, n_cols))
            
			# Create list to store rowspan values 
            skip_index = [0 for i in range(0, n_cols)]
			
            # Start by iterating over each row in this table...
            row_counter = 0
            for row in self.tables_html[n].find_all("tr"):
                
                # Skip row if it's blank
                if len(row.find_all(["td", "th"])) == 0:
                    next
                
                else:
                    
                    # Get all cells containing data in this row
                    columns = row.find_all(["td", "th"])
                    col_dim = []
                    row_dim = []
                    col_dim_counter = -1
                    row_dim_counter = -1
                    col_counter = -1
                    this_skip_index = copy.deepcopy(skip_index)
                    
                    for col in columns:
                        
                        # Determine cell dimensions
                        colspan = col.get("colspan")
                        if colspan is None:
                            col_dim.append(1)
                        else:
                            col_dim.append(int(colspan))
                        col_dim_counter += 1
                            
                        rowspan = col.get("rowspan")
                        if rowspan is None:
                            row_dim.append(1)
                        else:
                            row_dim.append(int(rowspan))
                        row_dim_counter += 1
                            
                        # Adjust column counter
                        if col_counter == -1:
                            col_counter = 0  
                        else:
                            col_counter = col_counter + col_dim[col_dim_counter - 1]
                            
                        while skip_index[col_counter] > 0:
                            col_counter += 1

                        # Get cell contents  
                        cell_data = col.get_text()
                        
                        # Insert data into cell
                        df.iat[row_counter, col_counter] = cell_data

                        # Record column skipping index
                        if row_dim[row_dim_counter] > 1:
                            this_skip_index[col_counter] = row_dim[row_dim_counter]
                
                # Adjust row counter 
                row_counter += 1
                
                # Adjust column skipping index
                skip_index = [i - 1 if i > 0 else i for i in this_skip_index]

            # Append dataframe to list of tables
            self.tables.append(df)
        
        return(self.tables)

In [3]:
#mps staff
url1 = "https://publications.parliament.uk/pa/cm/cmsecret/sponsor-01.htm"
url2 = "https://publications.parliament.uk/pa/cm/cmsecret/sponsor-02.htm"
url3 = "https://publications.parliament.uk/pa/cm/cmsecret/sponsor-03.htm"
url4 = "https://publications.parliament.uk/pa/cm/cmsecret/sponsor-04.htm"
url5 = "https://publications.parliament.uk/pa/cm/cmsecret/sponsor-05.htm"
ssa1 = html_tables(url1)
ssa2 = html_tables(url2)
ssa3 = html_tables(url3)
ssa4 = html_tables(url4)
ssa5 = html_tables(url5)
ssa1.read()[0].to_csv("ssa1.csv", header = False, index = False)
ssa2.read()[0].to_csv("ssa2.csv", header = False, index = False)
ssa3.read()[0].to_csv("ssa3.csv", header = False, index = False)
ssa4.read()[0].to_csv("ssa4.csv", header = False, index = False)
ssa5.read()[0].to_csv("ssa5.csv", header = False, index = False)


  
 


    #ssa_url = "https://publications.parliament.uk/pa/cm/cmsecret/sponsor-01.htm"#
#ssa = html_tables(ssa_url)
#wb = load_workbook('GW.xlsx')
#MPintsh=wb["MPint"]
#df=pd.DataFrame(ssa.read())
#MPintsh.append(ssa.read())
#with ExcelWriter('GW.xlsx') as writer:
   # df.to_excel(writer, sheet_name='MPint')


In [4]:
#defining excel sheets
wb = load_workbook('GW.xlsx')
Lordstaffsh=wb["Lordstaff"]

In [7]:
from string import ascii_lowercase
urls=[]
etw=[]
for c in ascii_lowercase:
    url="https://www.parliament.uk/mps-lords-and-offices/standards-and-financial-interests/house-of-lords-commissioner-for-standards-/register-of-lords-interests/register-of-interests-of-lords-members-staff-by-members-name/?letter={}".format(c)
    page=requests.get(url).content
    soup = BeautifulSoup(page, 'html.parser')
    if soup.text.find('No results found for that letter') != -1:
        continue
    else:
        urls.append(soup)
        print(url)
        names = [names.string for names in soup.find_all('h2')]
        names.pop()
        etw.append(names)
print(etw)
    

https://www.parliament.uk/mps-lords-and-offices/standards-and-financial-interests/house-of-lords-commissioner-for-standards-/register-of-lords-interests/register-of-interests-of-lords-members-staff-by-members-name/?letter=a
https://www.parliament.uk/mps-lords-and-offices/standards-and-financial-interests/house-of-lords-commissioner-for-standards-/register-of-lords-interests/register-of-interests-of-lords-members-staff-by-members-name/?letter=b
https://www.parliament.uk/mps-lords-and-offices/standards-and-financial-interests/house-of-lords-commissioner-for-standards-/register-of-lords-interests/register-of-interests-of-lords-members-staff-by-members-name/?letter=c
https://www.parliament.uk/mps-lords-and-offices/standards-and-financial-interests/house-of-lords-commissioner-for-standards-/register-of-lords-interests/register-of-interests-of-lords-members-staff-by-members-name/?letter=d
https://www.parliament.uk/mps-lords-and-offices/standards-and-financial-interests/house-of-lords-commiss

In [8]:

titles=["Rt", "Hon", "Sir", "Mr", "Mrs", "Ms", "Dr", "The", "the", "Rt", "Hon.", "Sir", "Lord", "Viscount", "Earl", "Archbishop","Baroness"]
suffixes=["MP", "QC","OBE","CBE", "CH"]
for i in range(len(urls)):
    abc = urls[i].find_all('table')
    trs=[]
    smt=[]
    for a in abc:
        b=a.find_all('tr')[1:]
        trs.append(b)
    for index,onelord in enumerate(trs):
        nameofthelord=etw[i][index]
        finalt=[]
        for onestaff in onelord:
            nameandjob=onestaff.find_all("td")
            #print(topu)
            pri=[str(nameofthelord)]
            for waz in nameandjob:
                cam=waz.string.strip()
                pri.append(cam)
                tame=pri[1]
                fir=[]
                sec=[]
                tit=[]
                suf=[]
                namesplit=[]
                splitnam=tame.split(" ")
                for snam in splitnam:
                    if snam in titles:
                        tit.append(snam)
                    elif snam in suffixes:
                        suf.append(snam)
                    else:
                        namesplit.append(snam)
                for index, namez in enumerate(namesplit):
                    if index==0:
                        fir.append(namez)
                    else:
                        sec.append(namez)
                full=[" ".join(tit), " ".join(fir), " ".join(sec), " ".join(suf)]
            pri.extend(full)
            Lordstaffsh.append(pri)
                #pri.append()
            #print(pri)
            finalt.append(pri)
        #print(finalt)
    
        smt.append(finalt)
           
    #print(smt)
            
         
#todo parser for name
#add lord


wb.save('GW.xlsx')


In [3]:
url = "https://www.parliament.uk/mps-lords-and-offices/standards-and-financial-interests/house-of-lords-commissioner-for-standards-/register-of-lords-interests/register-of-interests-of-lords-members-staff-by-members-name/?letter=A"
page=requests.get(url).content
soup = BeautifulSoup(page, 'html.parser')



names = [names.string for names in soup.find_all('h2')]
names.pop()
print(names)

['Lord Addington', 'Lord Adonis', 'Baroness Afshar', 'Lord Ahmed', 'Lord Alderdice', 'Lord Allen of Kensington', 'Lord Alli', 'Baroness Altmann', 'Lord Alton of Liverpool', 'Lord Anderson of Swansea', 'Lord Archer of Weston-Super-Mare', 'Lord Astor of Hever']
